In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, f1_score, accuracy_score, log_loss
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import xgboost as xbg
from sklearn.pipeline import Pipeline
import catboost as cat

In [4]:
train_data = pd.read_csv(r'C:\Users\user\OneDrive\Documents\code\zindi\churn\Train.csv')
test_data = pd.read_csv(r'C:\Users\user\OneDrive\Documents\code\zindi\churn\Test.csv') 

In [5]:
train_data.head(3)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,NaN,NaN,NO,62,All-net 500F=2000F;5d,35.0,0
1,71c44b5ba328db5c4192a80f7cf8f244d9350ed0,NaN,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,NaN,2.0,NO,40,"Data: 100 F=40MB,24H",22.0,0
2,ce46411b1526c94f20a383b8cb188f8d27f82a0a,TAMBACOUNDA,K > 24 month,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,NaN,NaN,NO,32,All-net 500F=2000F;5d,3.0,0


In [6]:
test_data.head(3)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK
0,af900d87e73b7ff6509d2203df4704a98aa5f2a6,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,1,NaN,NaN
1,5335efd940280b82143272275637d1e65d37eadb,NaN,K > 24 month,NaN,NaN,10.0,3.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NO,2,NaN,NaN
2,a581f4fa08677c26f83f643248c667e241043086,NaN,K > 24 month,1900.0,15.0,2299.0,766.0,21.0,414.0,NaN,7.0,2.0,NaN,NaN,NO,27,"Data: 100 F=40MB,24H",17.0


In [7]:
user_id = test_data['user_id']
train_y = train_data['CHURN']

In [8]:
train_data = train_data.drop(['user_id', 'CHURN','MRG','ZONE1', 'ZONE2'], axis =1)
test_data = test_data.drop(['user_id','MRG','ZONE1', 'ZONE2'], axis =1)

In [9]:
print(train_data.isnull().sum(),
test_data.isnull().sum())

REGION            157520
TENURE                 0
MONTANT           140277
FREQUENCE_RECH    140277
REVENUE           134663
ARPU_SEGMENT      134663
FREQUENCE         134663
DATA_VOLUME       196854
ON_NET            145819
ORANGE            166317
TIGO              239386
REGULARITY             0
TOP_PACK          167329
FREQ_TOP_PACK     167329
dtype: int64 REGION            39293
TENURE                0
MONTANT           34951
FREQUENCE_RECH    34951
REVENUE           33490
ARPU_SEGMENT      33490
FREQUENCE         33490
DATA_VOLUME       49338
ON_NET            36383
ORANGE            41200
TIGO              59788
REGULARITY            0
TOP_PACK          41703
FREQ_TOP_PACK     41703
dtype: int64


In [10]:
columns = train_data.columns
columnss = test_data.columns

In [11]:
miss = ['REGION','TOP_PACK']

for mi in miss:
    train_data[mi] = train_data[mi].fillna(train_data[mi].value_counts().index[0])
    test_data[mi] = test_data[mi].fillna(test_data[mi].value_counts().index[0])


In [12]:
train_data.head(3)

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK
0,SAINT-LOUIS,K > 24 month,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,62,All-net 500F=2000F;5d,35.0
1,DAKAR,K > 24 month,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,40,"Data: 100 F=40MB,24H",22.0
2,TAMBACOUNDA,K > 24 month,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,32,All-net 500F=2000F;5d,3.0


In [13]:
test_data.head(3)


,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK
0,DAKAR,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,All-net 500F=2000F;5d,NaN
1,DAKAR,K > 24 month,NaN,NaN,10.0,3.0,1.0,NaN,NaN,NaN,NaN,2,All-net 500F=2000F;5d,NaN
2,DAKAR,K > 24 month,1900.0,15.0,2299.0,766.0,21.0,414.0,NaN,7.0,2.0,27,"Data: 100 F=40MB,24H",17.0


In [14]:
train_data.isnull().sum()

REGION                 0
TENURE                 0
MONTANT           140277
FREQUENCE_RECH    140277
REVENUE           134663
ARPU_SEGMENT      134663
FREQUENCE         134663
DATA_VOLUME       196854
ON_NET            145819
ORANGE            166317
TIGO              239386
REGULARITY             0
TOP_PACK               0
FREQ_TOP_PACK     167329
dtype: int64

In [15]:
label = LabelEncoder()

In [16]:
cols = ['TENURE',  'TOP_PACK', 'REGION']

for col in cols:
    print(len(train_data[col].unique()))
    print(len(test_data[col].unique()))

8
8
111
92
14
14


In [17]:
# working on the categorical data
for col in cols:
    if len(train_data[col].unique()):
        train_data[col] = label.fit_transform(train_data[col])
        test_data[col] = label.fit_transform(test_data[col])
  
    
    

In [18]:
train_data.head(3)

,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK
0,9,7,17000.0,32.0,18000.0,6000.0,34.0,NaN,97.0,355.0,6.0,62,15,35.0
1,0,7,4300.0,29.0,4427.0,1476.0,37.0,1764.0,8.0,3.0,0.0,40,25,22.0
2,11,7,1500.0,3.0,1500.0,500.0,3.0,NaN,30.0,30.0,NaN,32,15,3.0


In [19]:
impute = SimpleImputer()

In [20]:
impu = ['ARPU_SEGMENT', 'FREQUENCE', 'DATA_VOLUME', 'ON_NET', 'ORANGE', 'TIGO','MONTANT', 'FREQUENCE_RECH', 'REVENUE','FREQ_TOP_PACK']

In [21]:
train_data = impute.fit_transform(train_data)
test_data = impute.fit_transform(test_data)

In [22]:
train_data = pd.DataFrame(train_data, columns = columns)
test_data = pd.DataFrame(test_data, columns = columnss)

In [23]:
train_x, test_x, train_y, test_y = train_test_split(train_data, train_y, test_size = 0.25,stratify = train_y, random_state = 42)

In [25]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(ran, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(train_x, train_y)

Fitting 3 folds for each of 500 candidates, totalling 1500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 49.7min
exception calling callback for <Future at 0x1b41611afa0 state=finished raised TerminatedWorkerError>
Traceback (most recent call last):
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 625, in _invoke_callbacks
    callback(self)
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 347, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 780, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 847, in dispatch_one_batch
    self._dispatch(tasks)
  File "C:\Users\user\anaconda3\lib\site-packages\joblib\parallel.py", line 765, in _dispatch
    job = self._backend.apply_async(batch, callback=cb)
  File "C:\Users\user\anacon

In [24]:
#xgb = xbg.XGBClassifier(objective ='binary:logistic', colsample_bytree = 0.3, learning_rate = 0.03,
                #max_depth = 7, alpha = 10, n_estimators = 500)
ran = RandomForestClassifier()
#cat = cat.CatBoostClassifier(learning_rate = 0.01,n_estimators = 500 )

In [35]:
xgb.fit(train_x, train_y)

XGBClassifier(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.03, max_delta_step=0, max_depth=7,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=500, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=10, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [125]:
ran.fit(train_x, train_y)

RandomForestClassifier(n_estimators=1000)

In [36]:
cat.fit(train_x, train_y)

Learning rate set to 0.11767
0:	learn: 0.5535531	total: 430ms	remaining: 7m 9s
1:	learn: 0.4603937	total: 622ms	remaining: 5m 10s
2:	learn: 0.4033590	total: 771ms	remaining: 4m 16s
3:	learn: 0.3674861	total: 929ms	remaining: 3m 51s
4:	learn: 0.3439246	total: 1.06s	remaining: 3m 31s
5:	learn: 0.3256438	total: 1.23s	remaining: 3m 24s
6:	learn: 0.3122271	total: 1.37s	remaining: 3m 14s
7:	learn: 0.3028601	total: 1.55s	remaining: 3m 12s
8:	learn: 0.2963675	total: 1.68s	remaining: 3m 4s
9:	learn: 0.2926578	total: 1.83s	remaining: 3m 1s
10:	learn: 0.2894285	total: 1.99s	remaining: 2m 58s
11:	learn: 0.2872552	total: 2.14s	remaining: 2m 56s
12:	learn: 0.2847601	total: 2.28s	remaining: 2m 53s
13:	learn: 0.2832508	total: 2.44s	remaining: 2m 51s
14:	learn: 0.2817869	total: 2.59s	remaining: 2m 50s
15:	learn: 0.2810199	total: 2.73s	remaining: 2m 47s
16:	learn: 0.2800284	total: 2.86s	remaining: 2m 45s
17:	learn: 0.2791741	total: 3.01s	remaining: 2m 44s
18:	learn: 0.2788277	total: 3.16s	remaining: 2m 

158:	learn: 0.2712833	total: 26.2s	remaining: 2m 18s
159:	learn: 0.2712683	total: 26.3s	remaining: 2m 18s
160:	learn: 0.2712468	total: 26.5s	remaining: 2m 18s
161:	learn: 0.2712315	total: 26.7s	remaining: 2m 17s
162:	learn: 0.2712205	total: 26.8s	remaining: 2m 17s
163:	learn: 0.2712076	total: 27s	remaining: 2m 17s
164:	learn: 0.2711962	total: 27.1s	remaining: 2m 17s
165:	learn: 0.2711854	total: 27.3s	remaining: 2m 16s
166:	learn: 0.2711542	total: 27.4s	remaining: 2m 16s
167:	learn: 0.2711430	total: 27.6s	remaining: 2m 16s
168:	learn: 0.2711104	total: 27.8s	remaining: 2m 16s
169:	learn: 0.2710906	total: 28s	remaining: 2m 16s
170:	learn: 0.2710650	total: 28.1s	remaining: 2m 16s
171:	learn: 0.2710439	total: 28.3s	remaining: 2m 16s
172:	learn: 0.2710253	total: 28.4s	remaining: 2m 15s
173:	learn: 0.2710157	total: 28.5s	remaining: 2m 15s
174:	learn: 0.2709926	total: 28.7s	remaining: 2m 15s
175:	learn: 0.2709652	total: 28.9s	remaining: 2m 15s
176:	learn: 0.2709527	total: 29s	remaining: 2m 15s

316:	learn: 0.2685321	total: 51s	remaining: 1m 49s
317:	learn: 0.2685138	total: 51.1s	remaining: 1m 49s
318:	learn: 0.2684906	total: 51.3s	remaining: 1m 49s
319:	learn: 0.2684761	total: 51.5s	remaining: 1m 49s
320:	learn: 0.2684648	total: 51.7s	remaining: 1m 49s
321:	learn: 0.2684447	total: 51.8s	remaining: 1m 49s
322:	learn: 0.2684335	total: 52s	remaining: 1m 48s
323:	learn: 0.2684169	total: 52.2s	remaining: 1m 48s
324:	learn: 0.2684051	total: 52.4s	remaining: 1m 48s
325:	learn: 0.2683889	total: 52.5s	remaining: 1m 48s
326:	learn: 0.2683693	total: 52.7s	remaining: 1m 48s
327:	learn: 0.2683552	total: 52.9s	remaining: 1m 48s
328:	learn: 0.2683355	total: 53.1s	remaining: 1m 48s
329:	learn: 0.2683226	total: 53.3s	remaining: 1m 48s
330:	learn: 0.2683196	total: 53.4s	remaining: 1m 47s
331:	learn: 0.2683072	total: 53.6s	remaining: 1m 47s
332:	learn: 0.2682932	total: 53.8s	remaining: 1m 47s
333:	learn: 0.2682786	total: 53.9s	remaining: 1m 47s
334:	learn: 0.2682631	total: 54.1s	remaining: 1m 4

472:	learn: 0.2663286	total: 1m 16s	remaining: 1m 24s
473:	learn: 0.2663168	total: 1m 16s	remaining: 1m 24s
474:	learn: 0.2663049	total: 1m 16s	remaining: 1m 24s
475:	learn: 0.2662920	total: 1m 16s	remaining: 1m 24s
476:	learn: 0.2662761	total: 1m 16s	remaining: 1m 23s
477:	learn: 0.2662583	total: 1m 16s	remaining: 1m 23s
478:	learn: 0.2662380	total: 1m 16s	remaining: 1m 23s
479:	learn: 0.2662211	total: 1m 17s	remaining: 1m 23s
480:	learn: 0.2662104	total: 1m 17s	remaining: 1m 23s
481:	learn: 0.2661926	total: 1m 17s	remaining: 1m 23s
482:	learn: 0.2661793	total: 1m 17s	remaining: 1m 22s
483:	learn: 0.2661691	total: 1m 17s	remaining: 1m 22s
484:	learn: 0.2661484	total: 1m 17s	remaining: 1m 22s
485:	learn: 0.2661345	total: 1m 17s	remaining: 1m 22s
486:	learn: 0.2661245	total: 1m 18s	remaining: 1m 22s
487:	learn: 0.2661111	total: 1m 18s	remaining: 1m 22s
488:	learn: 0.2660938	total: 1m 18s	remaining: 1m 21s
489:	learn: 0.2660827	total: 1m 18s	remaining: 1m 21s
490:	learn: 0.2660688	total:

626:	learn: 0.2643338	total: 1m 38s	remaining: 58.6s
627:	learn: 0.2643160	total: 1m 38s	remaining: 58.5s
628:	learn: 0.2643043	total: 1m 38s	remaining: 58.3s
629:	learn: 0.2642928	total: 1m 38s	remaining: 58.1s
630:	learn: 0.2642871	total: 1m 39s	remaining: 58s
631:	learn: 0.2642862	total: 1m 39s	remaining: 57.8s
632:	learn: 0.2642709	total: 1m 39s	remaining: 57.6s
633:	learn: 0.2642486	total: 1m 39s	remaining: 57.5s
634:	learn: 0.2642264	total: 1m 39s	remaining: 57.3s
635:	learn: 0.2642096	total: 1m 39s	remaining: 57.2s
636:	learn: 0.2641998	total: 1m 40s	remaining: 57s
637:	learn: 0.2641859	total: 1m 40s	remaining: 56.8s
638:	learn: 0.2641726	total: 1m 40s	remaining: 56.7s
639:	learn: 0.2641588	total: 1m 40s	remaining: 56.5s
640:	learn: 0.2641543	total: 1m 40s	remaining: 56.3s
641:	learn: 0.2641439	total: 1m 40s	remaining: 56.2s
642:	learn: 0.2641264	total: 1m 40s	remaining: 56s
643:	learn: 0.2641066	total: 1m 41s	remaining: 55.9s
644:	learn: 0.2640962	total: 1m 41s	remaining: 55.7s

784:	learn: 0.2623776	total: 2m 3s	remaining: 33.8s
785:	learn: 0.2623587	total: 2m 3s	remaining: 33.6s
786:	learn: 0.2623471	total: 2m 3s	remaining: 33.5s
787:	learn: 0.2623345	total: 2m 3s	remaining: 33.3s
788:	learn: 0.2623278	total: 2m 3s	remaining: 33.1s
789:	learn: 0.2623199	total: 2m 4s	remaining: 33s
790:	learn: 0.2622988	total: 2m 4s	remaining: 32.8s
791:	learn: 0.2622957	total: 2m 4s	remaining: 32.6s
792:	learn: 0.2622760	total: 2m 4s	remaining: 32.5s
793:	learn: 0.2622647	total: 2m 4s	remaining: 32.3s
794:	learn: 0.2622566	total: 2m 4s	remaining: 32.2s
795:	learn: 0.2622505	total: 2m 4s	remaining: 32s
796:	learn: 0.2622439	total: 2m 4s	remaining: 31.8s
797:	learn: 0.2622350	total: 2m 5s	remaining: 31.7s
798:	learn: 0.2622230	total: 2m 5s	remaining: 31.5s
799:	learn: 0.2622029	total: 2m 5s	remaining: 31.4s
800:	learn: 0.2621866	total: 2m 5s	remaining: 31.2s
801:	learn: 0.2621734	total: 2m 5s	remaining: 31s
802:	learn: 0.2621635	total: 2m 5s	remaining: 30.9s
803:	learn: 0.2621

941:	learn: 0.2605864	total: 2m 26s	remaining: 9.05s
942:	learn: 0.2605752	total: 2m 27s	remaining: 8.89s
943:	learn: 0.2605687	total: 2m 27s	remaining: 8.74s
944:	learn: 0.2605553	total: 2m 27s	remaining: 8.58s
945:	learn: 0.2605495	total: 2m 27s	remaining: 8.42s
946:	learn: 0.2605362	total: 2m 27s	remaining: 8.27s
947:	learn: 0.2605221	total: 2m 27s	remaining: 8.11s
948:	learn: 0.2605130	total: 2m 27s	remaining: 7.95s
949:	learn: 0.2605098	total: 2m 28s	remaining: 7.79s
950:	learn: 0.2604945	total: 2m 28s	remaining: 7.64s
951:	learn: 0.2604897	total: 2m 28s	remaining: 7.48s
952:	learn: 0.2604788	total: 2m 28s	remaining: 7.33s
953:	learn: 0.2604673	total: 2m 28s	remaining: 7.17s
954:	learn: 0.2604532	total: 2m 28s	remaining: 7.01s
955:	learn: 0.2604461	total: 2m 28s	remaining: 6.86s
956:	learn: 0.2604394	total: 2m 29s	remaining: 6.7s
957:	learn: 0.2604322	total: 2m 29s	remaining: 6.55s
958:	learn: 0.2604195	total: 2m 29s	remaining: 6.39s
959:	learn: 0.2604117	total: 2m 29s	remaining: 

In [37]:
#vals = xgb.predict(test_x)
valss = ran.predict(test_x)
#valsss = cat.predict(test_x)

In [38]:
classification_report(test_y, vals)

'              precision    recall  f1-score   support\n\n           0       0.91      0.93      0.92     81289\n           1       0.68      0.61      0.64     18711\n\n    accuracy                           0.87    100000\n   macro avg       0.79      0.77      0.78    100000\nweighted avg       0.87      0.87      0.87    100000\n'

In [128]:
classification_report(test_y, valss)

'              precision    recall  f1-score   support\n\n           0       0.91      0.93      0.92     81289\n           1       0.66      0.62      0.64     18711\n\n    accuracy                           0.87    100000\n   macro avg       0.79      0.77      0.78    100000\nweighted avg       0.87      0.87      0.87    100000\n'

In [39]:
classification_report(test_y, valsss)

'              precision    recall  f1-score   support\n\n           0       0.91      0.93      0.92     81289\n           1       0.67      0.61      0.64     18711\n\n    accuracy                           0.87    100000\n   macro avg       0.79      0.77      0.78    100000\nweighted avg       0.87      0.87      0.87    100000\n'

In [40]:
print(f1_score(test_y, vals))
print(accuracy_score(test_y, vals))
print(confusion_matrix(test_y, vals))
print(roc_auc_score(test_y, vals))
print(log_loss(test_y, vals))

0.6411720046115343
0.87239
[[75838  5451]
 [ 7310 11401]]
0.7711318385217135
4.407536841810474


In [130]:
print(f1_score(test_y, valss))
print(accuracy_score(test_y, valss))
print(confusion_matrix(test_y, valss))
print(roc_auc_score(test_y, valss))
print(log_loss(test_y, valss))

0.6385727637583154
0.86852
[[75237  6052]
 [ 7096 11615]]
0.7731537126671906
4.541206712039332


In [41]:
print(f1_score(test_y, valsss))
print(accuracy_score(test_y, valsss))
print(confusion_matrix(test_y, valsss))
print(roc_auc_score(test_y, valsss))
print(log_loss(test_y, valsss))

0.6396406513194833
0.87164
[[75772  5517]
 [ 7319 11392]]
0.7704853792953728
4.43344145184096


In [42]:
#test_val = xgb.predict_proba(test_data)[:,1]
test_vals = ran.predict_proba(test_data)[:,1]
#test_valss = cat.predict_proba(test_data)[:,1]

In [43]:
#SUBMISSION AREA

sub = {'user_id': user_id, 'CHURN': test_val}
ans = pd.DataFrame(sub)
ans.to_csv('submission9.csv', index =False)

In [133]:
subs = {'user_id': user_id, 'CHURN': test_vals}
ans = pd.DataFrame(subs)
ans.to_csv('submission10.csv', index =False)

In [44]:
subss = {'user_id': user_id, 'CHURN': test_valss}
ans = pd.DataFrame(subss)
ans.to_csv('submission11.csv', index =False)